# Manual App Authentication

In this tutorial we will use the [SpatiaFi Auth API](https://docs.spatiafi.com/auth/) directly to generate new app credentials.

The examples here are written in Python, but the same process can be used in any language or library that supports [OAuth 2.0 Client Credentials Grant](https://auth0.com/docs/flows/concepts/client-credentials). These credentials can then be used in your own applications to authenticate to the SpatiaFi API.

**If you are using the SpatiaFi API in Python or a Jupyter Notebook, you can skip this tutorial and use the [SpatiaFi Python Library](https://github.com/climateengine/py-spatiafi) instead.**

---

## Overview

We will first authenticate to the [SpatiaFi Auth API](https://auth.spatiafi.com/) once using our username and password.
Then using the returned access token we can generate new "App Credentials" as a `client_id` and `client_secret`.

Finally, we'll save the App Credentials to a file and load them into our environment variables.
We will then be able to use these app credentials to generate new access tokens in the future.

### Requirements

`requests` and `authlib` libraries will be used for code examples in this tutorial, to install:

In [ ]:
%pip install requests authlib

In [ ]:
import json
import time
from pprint import pprint

import requests
from authlib.integrations.requests_client import OAuth2Session
from authlib.oauth2.rfc7523 import ClientSecretJWT

## Basic Auth Token Endpoint

An access token to can be obtained by making a `GET` request to `https://auth.spatiafi.com/api/v1/auth/jwt/token/basic` with an [HTTP Basic Authentication](https://en.wikipedia.org/wiki/Basic_access_authentication) header.

This is straightforward using [requests](https://requests.readthedocs.io/en/latest/). Of course, remember to use your own username and password.

In [ ]:
url = "https://auth.spatiafi.com/api/v1/auth/jwt/token/basic"

username = "null@example.com"
password = "example_password123"

response = requests.get(url, auth=(username, password))

response_data = response.json()
access_token = response_data["access_token"]

pprint(response_data)

## Generate App Credentials

We now have an "access token", but **this token only lasts ~15 minutes**.  We can check the expiry time with:

In [ ]:
minutes, seconds = divmod(response_data["expires_at"] - int(time.time()), 60)

print(f"Access token expires in: {minutes} minutes, {seconds} seconds")

To work around this short expiry time, we generate App Credentials (`client_id` and `client_secret`) and use [authlib](https://docs.authlib.org/en/latest/) to obtain new access tokens.

When using App Credentials to obtain new access tokens, your password is not used, and your `client_secret` is used it _encrypt_ your request before sending.
This means that you can authenticate to the SpatiaFi API without transferring your password or `client_secret` over the internet.

### Create `client_id` and `client_secret`

In order to create App Credentials, we will make an empty `POST` request to `/api/v1/clients` endpoint, and include the `access_token` from above in the Header.

`client_secret` is only shown once during client creation and can't be accessed later, so make sure to save it somewhere safe.

In [ ]:
headers = {"Authorization": "Bearer " + access_token}

response = requests.post("https://auth.spatiafi.com/api/v1/clients/", headers=headers)
response_data = response.json()
print(f"status: {response.status_code}")
print(response_data)

client_id = response_data["client_id"]
client_secret = response_data["client_secret"]

### Save App Credentials
In order to use these credentials in the future we will save them to our local machine.

In [ ]:
file_location = "app_credentials.json"

with open(file_location, "w") as f:
    json.dump(response_data, f)
    print("App Credentials saved to:", file_location)

## Use App Credentials to generate a new Access Token

We can now use `authlib` and these credentials to generate access tokens:

In [ ]:
def get_session():
    client = OAuth2Session(
        client_id,
        client_secret,
        token_endpoint="https://auth.spatiafi.com/api/v1/auth/jwt/token",
        grant_type="client_credentials",
        token_endpoint_auth_method=ClientSecretJWT(
            "https://auth.spatiafi.com/api/v1/auth/jwt/token"
        ),
    )
    client.fetch_token()
    return client


session = get_session()
print("Getting an access_token with client_id and client_secret:\n")
response_data = session.fetch_token()
minutes, seconds = divmod(response_data["expires_at"] - int(time.time()), 60)

print(f"New access token expires in: {minutes} minutes, {seconds} seconds")

## Use in App Development

The `session` object can now be used to make requests to the SpatiaFi API, and will automatically refresh the access token when needed.

**Note:** If you are using the SpatiaFi API in Python or a Jupyter Notebook, you can skip this tutorial and use the [SpatiaFi Python Library](https://github.com/climateengine/py-spatiafi)